In [10]:
!pip install scikit-learn
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
train_path = "Dataset/train"
categories = [
    "Acne and Rosacea Photos",
    "Eczema Photos",
    "Hair Loss Photos Alopecia and other Hair Diseases",
    "Nail Fungus and other Nail Disease",
    "Scabies Lyme Disease and other Infestations and Bites",
]

In [12]:
images = []
labels = []

for idx, category in enumerate(categories):
    category_path = os.path.join(train_path, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
            img = cv2.resize(img, (180, 180))  
            images.append(img)
            labels.append(idx)

In [13]:
images = np.array(images) 
labels = np.array(labels)

In [14]:
num_classes = len(categories)
labels = to_categorical(labels, num_classes=num_classes)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [16]:
vgg_model = VGG19(weights="imagenet", include_top=False, input_shape=(180, 180, 3))

In [17]:
for layer in vgg_model.layers:
    layer.trainable = False

In [20]:
features_train = vgg_model.predict(x_train)
features_test = vgg_model.predict(x_test)

 3/95 ━━━━━━━━━━━━━━━━━━━━ 7:33 5s/step

KeyboardInterrupt: 

In [21]:
x_train_features = features_train.reshape(features_train.shape[0], -1)
x_test_features = features_test.reshape(features_test.shape[0], -1)

In [22]:
model = Sequential([
    Dense(200, activation="relu"),
    Dense(170, activation="relu"),
    Dense(num_classes, activation="softmax")
])

In [23]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [24]:
history = model.fit(
    x_train_features, y_train,
    validation_data=(x_test_features, y_test),
    epochs=100,
    batch_size=32
)

Epoch 1/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5535 - loss: 5.6897 - val_accuracy: 0.1744 - val_loss: 1.6114
Epoch 2/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7829 - loss: 0.7175 - val_accuracy: 0.2919 - val_loss: 1.5850
Epoch 3/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8699 - loss: 0.3968 - val_accuracy: 0.2748 - val_loss: 1.6344
Epoch 4/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9438 - loss: 0.2094 - val_accuracy: 0.1691 - val_loss: 1.6206
Epoch 5/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9539 - loss: 0.1483 - val_accuracy: 0.3950 - val_loss: 1.5413
Epoch 6/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9550 - loss: 0.1716 - val_accuracy: 0.3765 - val_loss: 1.5979
Epoch 7/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9679 - loss: 0.1243 - val_accuracy: 0.2655 - val_loss: 1.6225
Epoch 8/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9754 - loss: 0.0883 - val_accuracy: 0.

KeyboardInterrupt: 

In [25]:
loss, accuracy = model.evaluate(x_test_features, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2122 - loss: 1.6369 
Test Accuracy: 22.46%


In [26]:
model.save("skin_disease_model.h5")

In [27]:
def predict_skin_disease(image_path):
    # Load and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (180, 180))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # Extract features using VGG19
    img_features = vgg_model.predict(img)
    img_features = img_features.reshape(1, -1)

    # Make prediction
    pred = model.predict(img_features)[0]
    predicted_class = np.argmax(pred)
    return categories[predicted_class]

In [28]:
test_image_path = "Dataset/test/Acne and Rosacea Photos/07SteroidPerioral.jpg"
predicted_disease = predict_skin_disease(test_image_path)
print(f"Predicted Disease: {predicted_disease}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted Disease: Nail Fungus and other Nail Disease
